In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation, LinearTriInterpolator
from tqdm import tqdm

    
name = 'Model'
workspace = '../modelData/model'
mf_exe_name = '../modelData/exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()
headMin = head[head>-1e+10].min()
headMax = head[head>-1e+10].max()
headRshp = head.reshape((head.shape[0],head.shape[2]))

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:
# define vertex interpolation functions
def triInterpolation(zValue, xPoints, yPoints):
    triObj = Triangulation(xPoints, yPoints)
    fz = LinearTriInterpolator(triObj, zValue)
    return fz

def interpVerticesZ(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    layTri = triInterpolation(zValues, xyCentroid[:,0], xyCentroid[:,1])
    
    for index, vertice in enumerate(tqdm(xyVertices, desc="Working vertices elevations for %s"%zName)):
        
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if index in row2[1:]:
                    newZ = zValues[index2]
        interpVerticesList.append(newZ)
    interpVerticesArray = np.array(interpVerticesList)
    return interpVerticesArray

def interpVerticesHeads(zName, zValues, xyCentroid, xyVertices):
    interpVerticesList=[]
    #filtering over real heads
    realHeadIndexArray = (zValues>-1.00000000e+10).nonzero()[0]
    #reassigning arrays
    zValuesFiltered = zValues[realHeadIndexArray]
    xyCentroidFiltered = xyCentroid[realHeadIndexArray]
    verticesIndexFiltered = []
    for headCell in realHeadIndexArray:
        verticesIndexFiltered+=cell2dList[headCell][1:]
    verticesIndexFiltered = list(set(verticesIndexFiltered))
    #triangle interpolation function
    layTri = triInterpolation(zValuesFiltered, xyCentroidFiltered[:,0], xyCentroidFiltered[:,1])
    #empty array
    interpVerticesArray = np.ones([nvert])*-1.00000000e+10
    for vertexIndex in tqdm(verticesIndexFiltered, desc="Working vertices heads for %s"%zName):
        vertice = xyVertices[vertexIndex]
        newZ = layTri(vertice[0],vertice[1])
        if newZ.mask:
            for index2, row2 in enumerate(cell2dList):
                if vertexIndex in row2[1:]:
                    if zValues[index2]>-1.00000000e+10:
                        #print(zValues[index2])
                        newZ = zValues[index2]
                    
        interpVerticesArray[vertexIndex] = newZ
    return interpVerticesArray

In [3]:
# define model variables from flopy instanes
nlay = mfmodel.disv.nlay.array
nvert = mfmodel.disv.nvert.array
ncpl = mfmodel.disv.ncpl.array
modelCellZ = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array)) #cell Z in all layers
modelHeadZ = np.vstack([headRshp,headRshp[-1]])
# XY array for all cellvertices and centroids
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0] 
centroidXYArray = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc))[0]

# XYZ for cell centroid
centroidXYZDict = {}
for lay in range(nlay):
    centroidXYZDict[str(lay)] = np.dstack((mfmodel.disv.cell2d.array.xc,mfmodel.disv.cell2d.array.yc,(modelCellZ[lay] + modelCellZ[lay+1])/2))[0]

cell2dList = []
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[3:] if x is not None]
    cell2dList.append(vertexIndexList)        
    
# calculate cell vertex Z
print('\nCalculating vertices elevations values',flush=True)
modelVertZ = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertZ[i] = interpVerticesZ('Lay '+str(i),modelCellZ[i], centroidXYArray, verticesXYArray)
    
# calculate cell vertex head
print('\nCalculating vertices head values',flush=True)
modelVertHead = np.zeros((nlay+1,nvert))
for i in range(nlay+1):
    modelVertHead[i] = interpVerticesHeads('Lay '+str(i),modelHeadZ[i], centroidXYArray, verticesXYArray)
    
cell2dArray = np.hstack(cell2dList)


Calculating vertices elevations values


Working vertices elevations for Lay 6: 100%|██████████| 43707/43707 [00:14<00:00, 3051.56it/s]


Calculating vertices head values



Working vertices heads for Lay 6: 100%|██████████| 43707/43707 [00:12<00:00, 3362.12it/s]


### for model geometry

In [4]:
baseGridXYZ = np.hstack((verticesXYArray,np.zeros([nvert,1])))
baseGrid = pv.PolyData(baseGridXYZ,cell2dArray)

layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working geometries for model grid")):
    workingGrid = baseGrid.copy()
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    layerTubes[str(lay)] = workingVolume
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../vtk/ModelGrid.vtk')

#print('Working on the outer model shell')
workingGrid = baseGrid.copy()
workingVolume = workingGrid.extrude([0,0,1])
for vert in tqdm(range(nvert),desc=("Working on the outer model shell")):
    workingVolume.points[vert][2] = modelVertZ[0,vert]
    workingVolume.points[vert+nvert][2] = modelVertZ[nlay,vert]
workingVolume.save('../outputData/vtk/ModelShell.vtk')

Working geometries for model grid: 100%|██████████| 6/6 [00:39<00:00,  6.55s/it]
ERROR:root:Unable to open file: /home/jovyan/voronoiModflow6/vtk/ModelGrid.vtk
Working on the outer model shell: 100%|██████████| 43707/43707 [00:07<00:00, 6145.41it/s]


### for cell heads

In [5]:
layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working head cell for model grid")):
    workingGrid = baseGrid.copy()
    workingGrid.point_arrays['vertexHead'] = modelVertHead[lay]
    workingGrid.cell_arrays['cellHead'] = headRshp[lay]
    cellArray = workingGrid.cell_arrays.get_array('cellHead')
    tresArray = cellArray<=-1E+10
    workingGrid.remove_cells(tresArray)
    workingVolume = workingGrid.extrude([0,0,1])
    for vert in range(nvert):
        workingVolume.points[vert][2] = modelVertZ[lay,vert]
        workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
    workingVolume.cast_to_unstructured_grid()
    layerTubes[str(lay)] = workingVolume 
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../outputData/vtk/CellHeadsGrid.vtk')

Working head cell for model grid: 100%|██████████| 6/6 [00:39<00:00,  6.63s/it]
ERROR:root:Unable to open file: /home/jovyan/voronoiModflow6/vtk/CellHeadsGrid.vtk


In [6]:
modelContour = totalModelGrid.contour(isosurfaces=10,scalars='vertexHead',rng =(headMin, headMax), preference='cell')
modelContour.save('../outputData/vtk/CellHeadsContour.vtk')

### for water table

In [7]:
wtCellList =[] 
for index in range(ncpl):
    headCell = headRshp[:,index]
    wtCell = headCell[headCell>-1.00000000e+30][0]
    wtCellList.append(wtCell)
wtVertexList = interpVerticesZ('Water Table',wtCellList, centroidXYArray, verticesXYArray)

workingGrid = baseGrid.copy()
workingGrid.cell_arrays['cellWaterTable']=np.array(wtCellList)
workingGrid.point_arrays['vertexWaterTable']=np.array(wtVertexList)
for vert in range(nvert):
    workingGrid.points[vert][2] = wtVertexList[vert]
workingGrid.save('../outputData/vtk/WaterTable.vtk')

Working vertices elevations for Water Table: 100%|██████████| 43707/43707 [00:13<00:00, 3167.94it/s]


### for drain boundary

In [8]:
drnPkg = mfmodel.get_package('drn_0')
drnStp = drnPkg.stress_period_data.array[0]

layerTubes = {}
for lay in tqdm(range(nlay),desc=("Working drain cell for model grid")):
    workingGrid = baseGrid.copy()
    #workingGrid.point_arrays['vertexHead'] = modelVertHead[lay]
    drainCells = [i[1] for i in drnStp.cellid if i[0]==lay]
    actElev = [i.elev for i in drnStp if i.cellid[0]==lay]
    
    drainElevArray = np.ones(ncpl)*-1E+10
    drainElevArray[drainCells] = actElev
    workingGrid.cell_arrays['drainElev'] = drainElevArray
    cellArray = workingGrid.cell_arrays.get_array('drainElev')
    tresArray = cellArray<=-1E+10
    workingGrid.remove_cells(tresArray)
    #print(workingGrid)
    if workingGrid.n_cells>0:
        workingVolume = workingGrid.extrude([0,0,1])
        for vert in range(nvert):
            workingVolume.points[vert][2] = modelVertZ[lay,vert]
            workingVolume.points[vert+nvert][2] = modelVertZ[lay+1,vert]
        workingVolume.cast_to_unstructured_grid()
        layerTubes[str(lay)] = workingVolume 
layerBlocks = pv.MultiBlock(layerTubes)
totalModelGrid = layerBlocks.combine()
totalModelGrid.save('../outputData/vtk/Drains.vtk')

Working drain cell for model grid: 100%|██████████| 6/6 [00:07<00:00,  1.19s/it]
